In [ ]:
import torch
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import time
from model import Net
import sys
import pandas as pd

In [ ]:
t = int(time.time())
useCUDA = False
dataPath = "large_field_preprocessed_data.csv"
epochs = 20
batchSize = 64
modelPath = f"models/{t}"

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
if not useCUDA:
	device = torch.device('cpu')

In [ ]:
data = pd.read_csv(dataPath)
data = data.drop(columns= ['SERIAL', 'PERNUM', 'HHWT', 'CLUSTER', 'STRATA', 'PERWT'])

In [ ]:
dataTensor = torch.tensor(data.values)

print(dataTensor.shape)

In [ ]:

net = Net()

optimizer = optim.Adam(net.parameters(), lr =1e-4)

criterion = nn.MSELoss()

print("Epochs Started")

for epoch in range(epochs):
	running_loss = 0.0
	for i, data in enumerate(trainLoader):
		X, y = data
		X.to(device)
		y.to(device)
		net.zero_grad()
		output = net(X)
		loss = criterion(output, y)
		loss.backward()
		optimizer.step()
		running_loss += loss.item()
		if i % 1000 == 999:    # print every 1000 mini-batches
			print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 1000:.3f}')
			running_loss = 0.0

torch.save(net.state_dict(), modelPath)
torch.cuda.empty_cache()
